# telemarketing_sales project

In [1]:
import pandas as pd

In [2]:
# загружаю датасеты с продажами из csv
tm_sales1 = pd.read_csv('tm_sales_1.csv', sep=';') 
tm_sales2 = pd.read_csv('tm_sales_2.csv', sep=';')
tm_sales3 = pd.read_csv('tm_sales_3.csv', sep=';')
logs = pd.read_csv('prod_activations_logs.csv', sep=';')

In [3]:
tm_sales2.head()

,FILIAL_ID,SUBS_ID,PROD_ID,ACT_DTTM
0,3,id5416547,1743,17-03-2020 10:17
1,4,NaN,7421,12-03-2020 11:25
2,7,8362218,9879,05-03-2020 11:42
3,2,2185490,3210,16-03-2020 16:28


In [4]:
logs.head()

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM
1,id4651830,1954,20-03-2020 14:59,01-12-2020 00:00
5,id7646509,6431,19-03-2020 13:00,19-03-2020 13:03
2,id7461794,3310,20-03-2020 17:25,01-12-2020 00:00
3,id5416547,1743,17-03-2020 10:17,25-03-2020 11:00
4,id8238421,1859,01-03-2020 11:42,01-03-2020 11:43


In [5]:
# соединяю три датсета с продажами
tm_sales = pd.concat([tm_sales1, tm_sales2, tm_sales3])

In [6]:
tm_sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 2
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   SUBS_ID    9 non-null      object
 1   FILIAL_ID  10 non-null     int64 
 2   PROD_ID    10 non-null     int64 
 3   ACT_DTTM   10 non-null     object
dtypes: int64(2), object(2)
memory usage: 400.0+ bytes


Далее обрабатываю данные исходя из их особенностей:

- если в файле с продажами встречается строка без указанного SUBS_ID, она пропускается

In [7]:
# удаляю строки с пустыми значениями
tm_sales = tm_sales.dropna(subset=['SUBS_ID'])

- сотрудники телемаркетинга не всегда указывают полный id, если 'id' нет в начале SUBS_ID, то нужно его добавить

In [8]:
# функция для добавления 'id'
def add_id(series):
    if series.startswith('id'):
        return x
    else:
        y = 'id'+x
        return y

In [9]:
tm_sales.SUBS_ID = tm_sales.SUBS_ID.apply(add_id)

In [10]:
tm_sales

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,20-03-2020 14:59
1,id7646509,5,6431,19-03-2020 13:00
2,id7412683,4,3313,22-03-2020 17:25
0,id5416547,3,1743,17-03-2020 10:17
2,id8362218,7,9879,05-03-2020 11:42
3,id2185490,2,3210,16-03-2020 16:28
0,id5764122,3,1499,18-03-2020 15:44
1,id7642700,6,3020,15-03-2020 14:21
2,id1374509,2,5677,17-03-2020 11:48


- поля в файлах могут быть расположены абсолютно случайным образом, но названия полей статичны

In [11]:
# объединяю таблицы по столбцам SUBS_ID и PROD_ID
sales_with_logs = logs.merge(tm_sales, on = ['SUBS_ID','PROD_ID'])

In [12]:
sales_with_logs.head()

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM,FILIAL_ID,ACT_DTTM
0,id4651830,1954,20-03-2020 14:59,01-12-2020 00:00,1,20-03-2020 14:59
1,id7646509,6431,19-03-2020 13:00,19-03-2020 13:03,5,19-03-2020 13:00
2,id5416547,1743,17-03-2020 10:17,25-03-2020 11:00,3,17-03-2020 10:17
3,id2185490,3210,16-03-2020 16:28,01-12-2020 00:00,2,16-03-2020 16:28
4,id7642700,3020,15-03-2020 14:21,15-03-2020 23:42,6,15-03-2020 14:21


- продажа не засчитывается, если отключение (END_DTTM) произошло меньше чем через 5 минут после подключения (START_DTTM)

In [13]:
# проверю, в каком формате даты
sales_with_logs.dtypes

SUBS_ID       object
PROD_ID        int64
START_DTTM    object
END_DTTM      object
FILIAL_ID      int64
ACT_DTTM      object
dtype: object

In [14]:
# перевожу данные с датой и временем в соответсвующий формат, с явным указанием формата
sales_with_logs['START_DTTM'] = pd.to_datetime(sales_with_logs['START_DTTM'],format='%d-%m-%Y %H:%M')
sales_with_logs['END_DTTM'] = pd.to_datetime(sales_with_logs['END_DTTM'], format='%d-%m-%Y %H:%M')

In [15]:
sales_with_logs.dtypes

SUBS_ID               object
PROD_ID                int64
START_DTTM    datetime64[ns]
END_DTTM      datetime64[ns]
FILIAL_ID              int64
ACT_DTTM              object
dtype: object

In [16]:
sales_with_logs

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM,FILIAL_ID,ACT_DTTM
0,id4651830,1954,2020-03-20 14:59:00,2020-12-01 00:00:00,1,20-03-2020 14:59
1,id7646509,6431,2020-03-19 13:00:00,2020-03-19 13:03:00,5,19-03-2020 13:00
2,id5416547,1743,2020-03-17 10:17:00,2020-03-25 11:00:00,3,17-03-2020 10:17
3,id2185490,3210,2020-03-16 16:28:00,2020-12-01 00:00:00,2,16-03-2020 16:28
4,id7642700,3020,2020-03-15 14:21:00,2020-03-15 23:42:00,6,15-03-2020 14:21


In [17]:
# добавляю колонку difference (разница между временем подключения и отключения)
sales_with_logs = sales_with_logs.assign(difference = sales_with_logs.END_DTTM - sales_with_logs.START_DTTM)
sales_with_logs

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM,FILIAL_ID,ACT_DTTM,difference
0,id4651830,1954,2020-03-20 14:59:00,2020-12-01 00:00:00,1,20-03-2020 14:59,255 days 09:01:00
1,id7646509,6431,2020-03-19 13:00:00,2020-03-19 13:03:00,5,19-03-2020 13:00,0 days 00:03:00
2,id5416547,1743,2020-03-17 10:17:00,2020-03-25 11:00:00,3,17-03-2020 10:17,8 days 00:43:00
3,id2185490,3210,2020-03-16 16:28:00,2020-12-01 00:00:00,2,16-03-2020 16:28,259 days 07:32:00
4,id7642700,3020,2020-03-15 14:21:00,2020-03-15 23:42:00,6,15-03-2020 14:21,0 days 09:21:00


In [18]:
# сохраняю в результат продажи с difference больше 5 минут
true_sales_with_logs = sales_with_logs.loc[sales_with_logs['difference'] > pd.Timedelta(5, 'm')]
true_sales_with_logs

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM,FILIAL_ID,ACT_DTTM,difference
0,id4651830,1954,2020-03-20 14:59:00,2020-12-01 00:00:00,1,20-03-2020 14:59,255 days 09:01:00
2,id5416547,1743,2020-03-17 10:17:00,2020-03-25 11:00:00,3,17-03-2020 10:17,8 days 00:43:00
3,id2185490,3210,2020-03-16 16:28:00,2020-12-01 00:00:00,2,16-03-2020 16:28,259 days 07:32:00
4,id7642700,3020,2020-03-15 14:21:00,2020-03-15 23:42:00,6,15-03-2020 14:21,0 days 09:21:00
